## 訓練TourAgency的名稱，找出相關的可能遺漏詞

In [72]:
import jieba, word2vec, gensim, logging, scipy, uniout, re, time, os
from gensim.models import Word2Vec
from datetime import datetime

In [2]:
jieba.set_dictionary(r'D:\BigData\Python\Workspace\TextMining\Project\dictionary_for_semantic_analysis\wordDict\dict.txt.big.txt')
jieba.load_userdict(r'D:\BigData\Python\Workspace\TextMining\Project\dictionary_for_semantic_analysis\wordDict\dict_revised_total_cleared(UTF-8).txt.big.txt')
jieba.load_userdict(r'D:\BigData\Python\Workspace\TextMining\Project\dictionary_for_semantic_analysis\wordDict\combinedDict.txt')
jieba.load_userdict(r'D:\BigData\Python\Workspace\TextMining\Project\dictionary_for_semantic_analysis\wordDict\Adj_Cleared_by_Albert.txt')
jieba.load_userdict(r'D:\BigData\Python\Workspace\TextMining\Project\dictionary_for_semantic_analysis\wordDict\TourAgency_dict.txt')
jieba.load_userdict(r'D:\BigData\Python\Workspace\TextMining\Project\dictionary_for_semantic_analysis\wordDict\TourAgency_CheckDict.txt')
'''
載入TourAgency的相關詞彙，並且逐行讀取時，就做一次分詞。
最後加入到，word2VecList內作為Word2Vec的訓練詞料。
'''
word2VecList = list()
with open(r'D:\BigData\Python\Workspace\TextMining\Project\txtFiles\word2Vec_Material.txt', 'r') as dataRead:
        for lines in dataRead.readlines():
            stopWords = list(jieba.cut(lines))
            word2VecList.append(stopWords)

Building prefix dict from D:\BigData\Python\Workspace\TextMining\Project\dictionary_for_semantic_analysis\wordDict\dict.txt.big.txt ...
Loading model from cache c:\users\albert\appdata\local\temp\jieba.u644c1e3a7131d16659188b660829ecb9.cache
Loading model cost 0.824 seconds.
Prefix dict has been built succesfully.


In [8]:
model = Word2Vec(word2VecList, window = 5, min_count = 1, size = 200, workers = 8, sg = 1)
#開始訓練

In [9]:
model.save(r'D:\BigData\Python\Workspace\TextMining\Project\trainConquency\word2Vec_TourAgency.txt')
#將訓練結果存檔

In [12]:
'''
這邊是讀取訓練結果，可以不用再重新訓練。
使用most_similar這個方法做字詞的查詢，並且顯示前N個相似字詞。
'''
load_model = Word2Vec.load(r'D:\BigData\Python\Workspace\TextMining\Project\trainConquency\word2Vec_TourAgency.txt')
for word, weight in load_model.most_similar(u'可樂', topn = 100):
    print word, weight

東南 0.834696412086
華泰 0.808557629585
五福 0.805129885674
山富 0.80324113369
天喜 0.781612753868
良友 0.778480172157
鳳凰 0.776040434837
大興 0.772459030151
喜鴻 0.771706938744
上順 0.770225524902
歐雅 0.767493784428
雄獅 0.759990334511
燦星 0.754251539707
大榮 0.751216292381
享趣 0.741813063622
康福 0.740839183331
可樂旅遊 0.738535523415
品冠 0.737835168839
利百加 0.734591364861
鴻 0.734216988087
這三家 0.729698896408
一陽 0.729317784309
雄師 0.726827740669
看上 0.723782718182
桂冠 0.720477700233
易遊網 0.719404578209
團有 0.714683055878
鎖定 0.711707949638
旅遊社 0.710484802723
奧捷團 0.710164904594
的還 0.709778308868
南龍 0.709454536438
千進 0.709063529968
大興旅行社 0.708838224411
團還 0.708688497543
萬大 0.708354949951
團時 0.708165585995
德瑞團 0.707752108574
加利利 0.707691788673
高價位 0.707411825657
飛鳥 0.704204082489
太平洋旅行社 0.704154014587
五福旅遊 0.701421558857
團我 0.700844109058
富泰 0.699900269508
會選 0.699273824692
新進 0.699211359024
看中 0.698572695255
想選 0.698026657104
巨匠 0.697321772575
第二團 0.696698307991
名生 0.695167541504
去年底 0.694352507591
台達 0.693743288517
豐年 0.6937

## 從訓練結果讀取，找出有相關的詞彙並存檔

In [14]:
'''
共做了3次I/O的讀取與寫入，
前兩次皆是讀入後，丟入到要檢查的dict內。
其中一個是，TourAgency的全名與簡稱，放入most_similar做字詞查詢。
另外一個則是用兩個字的簡稱，如：東南，
利用正規表示法，將符合關鍵字的字詞，全部都抓取，最後寫入到檔案中。
通常KeyError是表示，文章內沒有該字詞，所以就直接跳過。
'''
with open(r'D:\BigData\Python\Workspace\TextMining\Project\dictionary_for_semantic_analysis\wordDict\tourAgency_Checked_Dict.txt', 'w') as tourAgencyWrite:
    wordInput = dict()
    check = dict()
    for tourAgency in open(r'D:\BigData\Python\Workspace\TextMining\Project\dictionary_for_semantic_analysis\wordDict\TourAgency_Dict.txt', 'r'):
        wordInput[tourAgency.strip()] = tourAgency.strip()
    for checkWord in open(r'D:\BigData\Python\Workspace\TextMining\Project\dictionary_for_semantic_analysis\wordDict\TourAgency_CheckDict.txt', 'r'):
        check[checkWord.strip()] = checkWord.strip()
    for tourAgency in wordInput:
        try:
            for word, weight in load_model.most_similar(u'%s' % tourAgency.strip().decode('utf-8'), topn = 10000000000):
                for simpleName in check:
                    if re.search(u'%s' % simpleName.decode('utf-8'), word) is not None:
                        tourAgencyWrite.write(word.encode('utf-8') + '\n')
                    else:
                        pass
        except:
            print 'KeyError happened!!'

KeyError happened!!
KeyError happened!!
KeyError happened!!
KeyError happened!!
KeyError happened!!
KeyError happened!!
KeyError happened!!
KeyError happened!!
KeyError happened!!
KeyError happened!!
KeyError happened!!
KeyError happened!!
KeyError happened!!
KeyError happened!!
KeyError happened!!
KeyError happened!!


In [18]:
count = 0
for word in open(r'D:\BigData\Python\Workspace\TextMining\Project\dictionary_for_semantic_analysis\wordDict\tourAgency_Checked_Dict.txt', 'r'):
    count += 1
print count

3252


In [19]:
'''
這邊則是做抓取後，重複的字詞查詢。
'''
check = dict()
with open(r'D:\BigData\Python\Workspace\TextMining\Project\dictionary_for_semantic_analysis\wordDict\tourAgency_Checked_Again_Dict.txt', 'w') as dataWrite:
    with open(r'D:\BigData\Python\Workspace\TextMining\Project\dictionary_for_semantic_analysis\wordDict\tourAgency_Checked_Dict.txt', 'r') as dataRead:
        for word in dataRead.readlines():
            if word.strip() not in check:
                check[word.strip()] = word.strip()
                dataWrite.write(check[word.strip()] + '\n')
            else:
                print check[word.strip()]

山富國際旅行社股份有限公司
鳳凰國際旅行社股份有限公司
太平洋地區
鳳凰山
長汎假期
利百加旅遊
喜泰
東南側
東南部
東南歐
太平洋戰爭
東南季風
泛太平洋
Ggogo
雄獅旅行社股份有限公司
飛鳥旅行社
可樂吧
百事可樂
東南岸
麗寶
東南亞地區
五福旅行社股份有限公司
長汎旅行社
東南西北
百順旅行社股份有限公司
百順旅遊
良友旅行社
金鳳凰
eztravel
鳳凰古城
長汎
上順旅行社
百威旅行社股份有限公司
山富旅遊
燦星旅遊
吉帝旅遊
東南旅遊
太平洋旅行社
飛鳥
百順
五福旅遊
良友
華泰
雄獅旅遊
百威
鳳凰旅行社
鳳凰旅遊
大興旅行社
可樂旅遊
東南亞
鳳凰
吉帝
百威旅遊
燦星
上順
易遊網
利百加
東南旅行社
五福
東南
太平洋
可樂
雄獅
大興
東南旅遊
可樂旅遊
上順旅行社
鳳凰旅遊
大興旅行社
雄獅旅遊
鳳凰
山富旅遊
華泰
百威
太平洋旅行社
長汎
五福旅遊
利百加旅遊
東南旅行社
燦星旅遊
良友旅行社
利百加
山富
雄獅
易遊網
燦星
百威旅遊
良友
百順
東南
大興
飛鳥旅行社
喜泰
太平洋地區
東南岸
吉帝旅遊
可樂
可樂吧
五福
吉帝
百順旅遊
上順
Ggogo
長汎假期
雄獅旅行社股份有限公司
飛鳥
上順旅行社股份有限公司
鳳凰古城
百事可樂
麗寶
鳳凰國際旅行社股份有限公司
長汎旅行社
東南季風
山富國際旅行社股份有限公司
太平洋
百威旅行社股份有限公司
東南西北
東南部
東南歐
東南亞
東南側
鳳凰山
eztravel
五福旅行社股份有限公司
東南亞地區
太平洋戰爭
百順旅行社股份有限公司
金鳳凰
泛太平洋
東南部
東南岸
太平洋地區
麗寶
東南西北
喜泰
雄獅旅行社股份有限公司
良友旅行社
可樂吧
東南側
東南亞地區
東南季風
長汎假期
上順旅行社股份有限公司
鳳凰山
東南歐
Ggogo
百事可樂
利百加旅遊
泛太平洋
吉帝旅遊
長汎
東南旅遊
太平洋旅行社
上順旅行社
長汎旅行社
燦星旅遊
山富旅遊
太平洋戰爭
鳳凰國際旅行社股份有限公司
山富國際旅行社股份有限公司
鳳凰古城
五福旅遊
飛鳥
華泰
良友
百順旅行社股份有限公司
五福旅行社股份有限公司
金鳳凰
雄獅旅遊
大興旅行社
百順旅遊
百威
eztravel
吉帝
可樂旅遊
東南亞
易遊網
燦星
鳳凰
上順
東南旅行社
五福
鳳凰旅行社
山富
利百加
鳳凰旅遊
雄

In [20]:
'''
檢查原詞庫是否重複，建立合併後的檢查詞庫。
'''
with open(r'D:\BigData\Python\Workspace\TextMining\Project\dictionary_for_semantic_analysis\wordDict\tourAgency_Combined_Dict.txt', 'w') as dataWrite:
    names = dict()
    for word_origin in open(r'D:\BigData\Python\Workspace\TextMining\Project\dictionary_for_semantic_analysis\wordDict\TourAgency_Dict.txt', 'r'):
        names[word_origin.decode('utf-8').strip()] = word_origin.decode('utf-8').strip()
    for word_checked in open(r'D:\BigData\Python\Workspace\TextMining\Project\dictionary_for_semantic_analysis\wordDict\tourAgency_Checked_Again_Dict.txt', 'r'):
        if word_checked not in names:
            names[word_checked.decode('utf-8').strip()] = word_checked.decode('utf-8').strip()
        else:
            print word_checked.decode('utf-8').strip()
    for name in names:
        dataWrite.write(name.encode('utf-8') + '\n')

## 文章分類

In [66]:
'''
先宣告建立放入key的list，以及放全部20家旅行社的文章的資料
最後型態會是{'旅行社名稱':['文章', '文章', '文章'...], '旅行社名稱':['文章', '文章', '文章'...]...}
再來宣告check這個list，是所有旅行社的全名以及別名，作為檢查的資料
第一個迴圈是先讀取20個key(旅行社名稱)，避免一直重複I/O的問題
第二個迴圈是開始讀取code(文章代碼)，達成自動化的部分
除了麗寶、易遊網、吉帝和可樂有另外的別名，
其他的旅行社皆是兩個字的簡稱就可以做為判斷，
如：東南旅遊簡稱是東南
'''
def classifyArt(keysPath, checkPath, codesPath):
    dataCount = 0
    keys = list()
    check = list()
    tourAgency = dict()
    for key in open(keysPath, 'r'):
        keys.append(key.strip().decode('utf-8'))
    for checkName in open(checkPath, 'r'):
        check.append(checkName.strip().decode('utf-8'))
    for code in open(codesPath, 'r'):
        try:
            '''
            第一個是Ptt的路徑，第二個是背包客棧的路徑
            '''
#             articlePath = r'D:\BigData\Project\Travel\ptt_TourAgency_evaluation\Japan\Data\TourAgency\CleanedData\Articles\%s_article.txt' % code.strip()
            articlePath = r'D:\BigData\Project\Travel\Backpackers\Japan\Data\TourAgency\CleanedData\Article\%s_article.txt' % code.strip()
            with open(articlePath, 'r') as articleRead:
                tempArticle = articleRead.read().decode('utf-8')
                for checkName in check:
                    articles = list()
                    if re.search(u'喜泰|華泰', tempArticle):
                        articles.append(tempArticle)
                        key = u'麗寶'
                        if key not in tourAgency.keys():
                            tourAgency[key] = articles
                        else:
                            tourAgency[key].extend(articles)
                    elif re.search(u'eztravel', tempArticle):
                        articles.append(tempArticle)
                        key = u'易遊網'
                        if key not in tourAgency.keys():
                            tourAgency[key] = articles
                        else:
                            tourAgency[key].extend(articles)
                    elif re.search(u'Ggogo', tempArticle):
                        articles.append(tempArticle)
                        key = u'吉帝'
                        if key not in tourAgency.keys():
                            tourAgency[key] = articles
                        else:
                            tourAgency[key].extend(articles)
                    elif re.search(u'康福', tempArticle):
                        articles.append(tempArticle)
                        key = u'可樂'
                        if key not in tourAgency.keys():
                            tourAgency[key] = articles
                        else:
                            tourAgency[key].extend(articles)
                    elif checkName in tempArticle:
                        articles.append(tempArticle)
                        for key in keys:
                            if key in checkName:
                                if key not in tourAgency.keys():
                                    tourAgency[key] = articles
                                else:
                                    tourAgency[key].extend(articles)
            dataCount += 1
        except:
            print '-' * 10 + code.strip() + ' is Error!!' + '-' * 10
    print '-' * 10 + 'All are done!!' + '-' * 10
    print 'Data Quantities: %i' % dataCount
    return tourAgency

In [56]:
#Ptt的分類
keysPath = r'D:\BigData\Python\Workspace\TextMining\Project\dictionary_for_semantic_analysis\wordDict\TourAgency_Dict_for_Key.txt'
checkPath = r'D:\BigData\Python\Workspace\TextMining\Project\dictionary_for_semantic_analysis\wordDict\tourAgency_Combined_Dict.txt'
codesPath = r'D:\BigData\Project\Travel\ptt_TourAgency_evaluation\Japan\Data\TourAgency\CleanedData\Codes\Codes.txt'
tourAgency = classifyArt(keysPath, checkPath, codesPath)

----------All are done!!----------
Data Quantities: 12370


In [67]:
#背包客棧的分類
keysPath = r'D:\BigData\Python\Workspace\TextMining\Project\dictionary_for_semantic_analysis\wordDict\TourAgency_Dict_for_Key.txt'
checkPath = r'D:\BigData\Python\Workspace\TextMining\Project\dictionary_for_semantic_analysis\wordDict\tourAgency_Combined_Dict.txt'
codesPath = r'D:\BigData\Project\Travel\Backpackers\Japan\Data\TourAgency\CleanedData\Codes\Codes.txt'
tourAgency = classifyArt(keysPath, checkPath, codesPath)

----------1502295&page_4 is Error!!----------
----------1502295&page_5 is Error!!----------
----------1502295&page_12 is Error!!----------
----------240186&page_2 is Error!!----------
----------829868&page_2 is Error!!----------
----------829868&page_3 is Error!!----------
----------866830&page_2 is Error!!----------
----------1350237&page_2 is Error!!----------
----------35927&page_2 is Error!!----------
----------35927&page_3 is Error!!----------
----------35927&page_4 is Error!!----------
----------35927&page_5 is Error!!----------
----------35927&page_6 is Error!!----------
----------298254&page_2 is Error!!----------
----------570289&page_2 is Error!!----------
----------1417061&page_2 is Error!!----------
----------221507&page_2 is Error!!----------
----------221507&page_3 is Error!!----------
----------3880&page_2 is Error!!----------
----------869736&page_2 is Error!!----------
----------869736&page_3 is Error!!----------
----------869736&page_4 is Error!!----------
----------8

In [64]:
'''
將分類完的文章依序寫入檔案中，
由tourAgency.keys()作為實際檔案寫入的key，
而fileName則是檢查key是否相符合用的，
檢查完畢後，以英文為檔案命名
fileName = '東南,setTour'
格式:savePath = r'D:\BigData\Python\Workspace\TextMining\Project\txtFiles\ptt_Articles'
'''
def writeToTxt(tourAgency, savePath, fileNamePath):
    for key in tourAgency.keys():
        for fileName in open(fileNamePath, 'r'):
            key_check = fileName.strip().split(',')[0].decode('utf-8')
            if key == key_check:
                newFileName = fileName.strip().split(',')[1].decode('utf-8')
                newSavePath = os.path.join(savePath, '%s.txt' % newFileName)
                with open(os.path.abspath(newSavePath), 'w') as dataWrite:
                    for article in tourAgency[key]:
                        dataWrite.write(article.encode('utf-8'))
    print 'All are done!!'

In [65]:
#Ptt的檔案寫入
savePath = r'D:\BigData\Python\Workspace\TextMining\Project\txtFiles\ptt_Articles'
fileNamePath = r'D:\BigData\Python\Workspace\TextMining\Project\dictionary_for_semantic_analysis\wordDict\TourAgency_Dict_for_Key_FileName.txt'
writeToTxt(tourAgency, savePath, fileNamePath)

All are done!!


In [68]:
#背包客棧的檔案寫入
savePath = r'D:\BigData\Python\Workspace\TextMining\Project\txtFiles\backpackers_Atricles'
fileNamePath = r'D:\BigData\Python\Workspace\TextMining\Project\dictionary_for_semantic_analysis\wordDict\TourAgency_Dict_for_Key_FileName.txt'
writeToTxt(tourAgency, savePath, fileNamePath)

All are done!!


## Ptt & Backpackers檔案合併

In [70]:
def combinedArticles(fileNamePath, savePath, readPttPath, readBackpackersPath):
    for fileName in open(fileNamePath, 'r'):
        try:
            newFileName = fileName.strip().split(',')[1].decode('utf-8')
            newSavePath = os.path.join(savePath, '%s.txt' % newFileName)
            with open(os.path.abspath(newSavePath), 'a') as articleWrite:
                newReadPttPath = os.path.join(readPttPath, '%s.txt' % newFileName)
                for pttArticle in open(os.path.abspath(newReadPttPath), 'r'):
                    articleWrite.write(pttArticle.decode('utf-8').encode('utf-8'))
                newReadBackpackersPath = os.path.join(readBackpackersPath, '%s.txt' % newFileName)
                for backpackersArticle in open(os.path.abspath(newReadBackpackersPath), 'r'):
                    articleWrite.write(backpackersArticle.decode('utf-8').encode('utf-8'))
        except:
            print '-' * 10 + key.strip() + ' is Error!!' + '-' * 10
    print 'All are done!!'

In [71]:
fileNamePath = r'D:\BigData\Python\Workspace\TextMining\Project\dictionary_for_semantic_analysis\wordDict\TourAgency_Dict_for_Key_FileName.txt'
savePath = r'D:\BigData\Python\Workspace\TextMining\Project\txtFiles\totalArticles'
readPttPath = r'D:\BigData\Python\Workspace\TextMining\Project\txtFiles\ptt_Articles'
readBackpackersPath = r'D:\BigData\Python\Workspace\TextMining\Project\txtFiles\backpackers_Atricles'
combinedArticles(fileNamePath, savePath, readPttPath, readBackpackersPath)

----------大興 is Error!!----------
----------大興 is Error!!----------
----------大興 is Error!!----------
----------大興 is Error!!----------
----------大興 is Error!!----------
All are done!!
